# 关键词提取

### 用途

* 用核心信息代表原始文档

* 在文本聚类、分类、自动摘要等领域中有着重要应用

### 需求：针对一篇文章，在不加人工干预的情况下提取关键词
### 当然，首先要进行分词
### 关键词分配： 事先给定关键词库，然后在文档中进行关键词检索
### 关键词提取：根据某种规则，从文档中抽取最重要的词作为关键词
* 有监督：抽取出候选词并标记是否为关键词，然后训练相应的模型

* 无监督：给词条打分，并基于最高分值抽取

## 无监督方式的分析思路——基于词频

### 分析思路1：按照词频高低进行提取

* 大量的高频词并无多少意义（停用词）

* 即使出现频率相同，常见词价值也明显低于不常见词

### 分析思路2：按照词条在文档中的重要性进行提取

* 如何确定词条在该文档中的重要性？

常见的方法：**TF-IDF、网络图**

## TF-IDF算法

### 信息检索（IR）中最常用的一种文本关键信息表示法

### 基本思想：
  
* 如果某个词在一篇文档中出现频率较高，并且在语料库中其他文本中出现频率较低，甚至不出现，则认为这个词具有很好的类别区分能力

### 词频TF：Term Frequency，衡量一个词在文档中出现的频率

* 平均而言出现越频繁的词，其重要性可能就越高

#### 考虑到文章长度的差异，需要对词频做标准化

* TF(w) = (w出现在文档中的次数)/(文档中的词的总数)

* TF(w) = (w出现在文档中的次数)/(文档中出现最多的词的次数)

### 逆文档频率IDF：Inverse Document Frequency，用于模拟在该语料库中，某一个词有多重要

* 有些词到处出现，但是明显是没有用的。比如各种停用词，过渡句用词等。

* 因此把罕见的词的重要性（weight）调高，把常见词的重要性调低

### IDF的具体算法

* IDF(w) = log(语料库中的文档总数/(含有该w的文档总数+1))

### TF-IDF = TF * IDF

* TF-IDF与一个词在文档中的出现次数成正比

* 与该词在整个语料中的出现次数成反比

### 优点
* 简单快速

* 结果也比较符合实际情况

### 缺点
* 单纯以“词频”横量一个词的重要性，不够全面，有时重要的词可能出现的次数并不多

* 无法考虑词与词之间的相互关系

* 这种算法无法体现词的位置信息，出现位置靠前的词与出现位置靠后的词，都被视为重要性相同，这是不正确的

  * 一种解决方式是，对全文的第一段和每一段的第一句话，给予较大的权重

## TF-IDF的具体实现

jieba, NLTK, sklearn, gensim等程序包都可以实现TF-IDF的计算。除算法细节上会有差异外，更多的是数据输入/输出格式上的不同。

### jieba

输出结果会自动按照TF-IDF值降序排列，并且直接给出的是词条而不是字典ID，便于阅读使用。

可在计算TF-IDF时直接完成分词，并使用停用词表和自定义词库，非常方便。

有默认的IDF语料库，可以不训练模型，直接进行计算。

以单个文本为单位进行分析。

> jieba.analyse.extract_tags(
> 
> sentence 为待提取的文本
> 
> topK = 20 : 返回几个 TF/IDF 权重最大的关键词
>
> withWeight = False : 是否一并返回关键词权重值
>
>allowPOS = () : 仅包括指定词性的词，默认值为空，即不筛选
>
>)

**jieba.analyse.set_idf_path(file_name)**

    关键词提取时使用自定义逆向文件频率（IDF）语料库 

> 劳动防护 13.900677652
>
> 生化学 13.900677652
>
> 奥萨贝尔 13.900677652
> 
> 奧薩貝爾 13.900677652
> 
> 考察队员 13.900677652

**jieba.analyse.set_stop_words(file_name)**

    关键词提取时使用自定义停止词（Stop Words）语料库 

**jieba.analyse.TFIDF(idf_path = None)**

    新建 TFIDF模型实例
    idf_path : 读取已有的TFIDF频率文件（即已有模型）
    使用该实例提取关键词：TFIDF实例.extract_tags()

In [3]:
import pandas as pd
# 有的环境配置下read_table出错，因此改用read_csv
raw = pd.read_csv("金庸-射雕英雄传txt精校版.txt",
                  names = ['txt'], sep ='aaa', encoding ="GBK" ,engine='python')
# 章节判断用变量预处理
def m_head(tmpstr):
    return tmpstr[:1]

def m_mid(tmpstr):
    return tmpstr.find("回 ")

raw['head'] = raw.txt.apply(m_head)
raw['mid'] = raw.txt.apply(m_mid)
raw['len'] = raw.txt.apply(len)
# raw['chap'] = 0
raw.head(0)
# 章节判断
chapnum = 0
for i in range(len(raw)):
    if raw['head'][i] == "第" and raw['mid'][i] > 0 and raw['len'][i] < 30 :
        chapnum += 1
    if chapnum >= 40 and raw['txt'][i] == "附录一：成吉思汗家族" :
        chapnum = 0
    raw.loc[i, 'chap'] = chapnum
    
# 删除临时变量
del raw['head']
del raw['mid']
del raw['len']
raw.head(0)
#提取章节
rawgrp = raw.groupby('chap')
chapter = rawgrp.sum()##.agg(sum) # 只有字符串列的情况下，sum函数自动转为合并字符串
chapter = chapter[chapter.index != 0]

In [4]:
import jieba
import jieba.analyse

# 注意：函数是在使用默认的TFIDF模型进行分析！
jieba.analyse.extract_tags(chapter.txt[1])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.879 seconds.
Prefix dict has been built succesfully.


['杨铁心',
 '包惜弱',
 '郭啸天',
 '颜烈',
 '丘处机',
 '武官',
 '杨二人',
 '官兵',
 '曲三',
 '金兵',
 '那道人',
 '道长',
 '娘子',
 '段天德',
 '咱们',
 '临安',
 '说道',
 '丈夫',
 '杨家枪',
 '两人']

In [5]:
jieba.analyse.extract_tags(chapter.txt[1], withWeight = True) # 要求返回权重值

[('杨铁心', 0.21886511509515091),
 ('包惜弱', 0.1685852913570757),
 ('郭啸天', 0.09908082913091291),
 ('颜烈', 0.05471627877378773),
 ('丘处机', 0.049556061537506184),
 ('武官', 0.04608486747703612),
 ('杨二人', 0.044305304110440376),
 ('官兵', 0.040144546232276104),
 ('曲三', 0.03439059290450272),
 ('金兵', 0.0336976598949901),
 ('那道人', 0.03117114380098961),
 ('道长', 0.02912588670625928),
 ('娘子', 0.026796070076125684),
 ('段天德', 0.025139911869037603),
 ('咱们', 0.023296768210644483),
 ('临安', 0.022991990912831523),
 ('说道', 0.022350916333591046),
 ('丈夫', 0.02221595763081643),
 ('杨家枪', 0.019765724469755074),
 ('两人', 0.0192267944114003)]

In [6]:
# 应用自定义词典改善分词效果
jieba.load_userdict('金庸小说词库.txt') # dict为自定义词典的路径

# 在TFIDF计算中直接应用停用词表
jieba.analyse.set_stop_words('停用词.txt')

TFres = jieba.analyse.extract_tags(chapter.txt[1], withWeight = True)
TFres[:10]

[('杨铁心', 0.24766315655503918),
 ('包惜弱', 0.19076756653563828),
 ('郭啸天', 0.11211778033234882),
 ('曲三', 0.06421085701511758),
 ('颜烈', 0.061915789138759794),
 ('丘处机', 0.05607659595033594),
 ('武官', 0.05214866582927771),
 ('杨二人', 0.0501349493881299),
 ('官兵', 0.045426723368101905),
 ('金兵', 0.03813156251275196)]

In [7]:
# 使用自定义TF-IDF频率文件
jieba.analyse.set_idf_path("idf.txt.big")
TFres1 = jieba.analyse.extract_tags(chapter.txt[1], withWeight = True)
TFres1[:10]

[('杨铁心', 0.24766315655503918),
 ('包惜弱', 0.19076756653563828),
 ('郭啸天', 0.11211778033234882),
 ('武官', 0.07028278767102464),
 ('颜烈', 0.061915789138759794),
 ('说道', 0.05856898972585386),
 ('丘处机', 0.05607659595033594),
 ('曲三', 0.05522219031294792),
 ('一个', 0.05354879060649496),
 ('杨二人', 0.05354879060649496)]

### sklearn

输出格式为矩阵，直接为后续的sklearn建模服务。

需要先使用背景语料库进行模型训练。

结果中给出的是字典ID而不是具体词条，直接阅读结果比较困难。

class sklearn.feature_extraction.text.TfidfTransformer()

发现参数基本上都不用动，所以这里就不介绍了..

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer

txtlist = [ " ".join(m_cut(w)) for w in chapter.txt.iloc[:5]] 

vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(txtlist) # 将文本中的词语转换为词频矩阵  

transformer = TfidfTransformer()  
tfidf = transformer.fit_transform(X)  #基于词频矩阵X计算TF-IDF值  
tfidf

NameError: name 'm_cut' is not defined

In [ ]:
tfidf.toarray() # 转换为数组

In [ ]:
tfidf.todense() # 转换为矩阵

In [ ]:
tfidf.todense().shape

In [ ]:
print("字典长度：", len(vectorizer.vocabulary_))
vectorizer.vocabulary_

## 实战练习

请使用《射雕》全文计算出jieba分词的IDF语料库，然后使用该语料库重新对第一章计算关键词。比较这样的分析结果和以前有何不同。

请自行编制将jieba分词的TF-IDF结果转换为文档-词条矩阵格式的程序。

请自行思考本章提供的三种TF-IDF实现方式的使用场景是什么。